In [1]:
import os
import pandas as pd
import utm
from pyproj import CRS
import hickle as hkl
from scipy.ndimage import zoom
import rasterio
from rasterio.transform import from_origin
import geoarray as ga
import numpy as np
from arosics import COREG_LOCAL
import geopandas as gpd
from shapely import Point


In [2]:
def get_crs(latitude, utminfo):
    if latitude>0:
        abool = False
    else: abool = True
    crs=CRS.from_dict({'proj':'utm', 'zone' : utminfo[2], 'south': abool})
    return crs


In [9]:
def make_points(easting, northing):
    firstx = easting + 5
    firsty = northing - 5
    yvals = []
    xvals = []
    def ypts(y):
        if y>firsty-270:
            y=y-10
            yvals.append(y)
            ypts(y)
    def xpts(x):
        if x<firstx+270:
            x=x+10
            xvals.append(x)
            xpts(x)
    ypts(firsty)
    yvals.append(firsty)
    xpts(firstx)
    xvals.append(firstx)
    coords = []
    
    coords = [str(Point(i, j)) for i, j in zip(xvals, yvals)]
    
    
    for i in range(len(xvals)):
        for j in range(len(yvals)):
            pts = Point(xvals[i], yvals[j])
            coords.append(pts)
    for i in range(len(coords)):
        coords[i] = str(coords[i])

    coords = gpd.GeoSeries.from_wkt(coords)
    table = pd.DataFrame()
    table['idx']= [idx]*len(coords)
    shp = gpd.GeoDataFrame(table, geometry=coords, crs=crs)
    outpath = os.path.join("corr_points", "points" + str(idx) + ".shp")
    shp.to_file(outpath)



In [10]:
os.chdir(r'C:\Users\Gautam.Mathur\coregout')
#os.mkdir("corr_points")
data = data = pd.read_csv(r"C:\Users\Gautam.Mathur\OneDrive - World Resources Institute\Gautam Intern Materials\chm-master-clean.csv")
for idx in range(1, 30):
    latitude = data.lat[idx]
    longitude = data.lon[idx]
    utminfo = utm.from_latlon(latitude, longitude)
    easting = utminfo[0]-140
    northing = utminfo[1]+140
    crs = get_crs(latitude, utminfo)
    make_points(easting, northing)
    